# OCC SERP time EDA

EDA en tiempo (serp)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
spark = SparkSession \
    .builder \
    .appName("occ_serp-time_eda") \
    .getOrCreate()

#filename = "occ-timeseries.jsonl"
filename = "*OCC_SERP_MX*jsonl*" #real: solo vacantes no agregadas

base_dir = f"harvester/occ_serp/"
df = spark.read.json(f"{base_dir}/{filename}")

In [3]:
df.printSchema()

root
 |-- crawler: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- __typename: string (nullable = true)
 |    |-- applied: boolean (nullable = true)
 |    |-- autoinclusion: struct (nullable = true)
 |    |    |-- __typename: string (nullable = true)
 |    |    |-- autoinclusion: boolean (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |-- bullets: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- __typename: string (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |-- category: struct (nullable = true)
 |    |    |-- __typename: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- rel: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |    |-- company: struct (nullable = true)
 |    |    |-- __typename: string 

In [4]:
df_c = df.withColumn("day", F.to_date(F.col("scraped_at"), "yyyy-MM-dd HH:mm:ss.SSSSSS"))
df_c.select('day').distinct().collect()

[Row(day=datetime.date(2023, 12, 22)),
 Row(day=datetime.date(2023, 12, 21)),
 Row(day=datetime.date(2023, 12, 20)),
 Row(day=datetime.date(2023, 12, 15)),
 Row(day=datetime.date(2023, 12, 17)),
 Row(day=datetime.date(2023, 12, 19))]

In [5]:
%%time
s1 = set(list(df_c.select(['data.id']).where("day='2023-12-21'").distinct().toPandas()['id']))
s2 = set(list(df_c.select(['data.id']).where("day='2023-12-22'").distinct().toPandas()['id']))
len(s2-s1)

CPU times: user 726 ms, sys: 82.1 ms, total: 809 ms
Wall time: 16.3 s


10101

In [6]:
%%time
df_c.select(["data.autoinclusion.autoinclusion", "data.id"]).groupby("autoinclusion").count().show()

+-------------+------+
|autoinclusion| count|
+-------------+------+
|         true|407112|
|        false|431668|
+-------------+------+

CPU times: user 2.94 ms, sys: 2.02 ms, total: 4.96 ms
Wall time: 6.95 s


In [8]:
s1 = set(list(df_c.filter("data.autoinclusion.autoinclusion = false").select(['data.id']).where("day='2023-12-22'").distinct().toPandas()['id']))
s2 = set(list(df_c.filter("data.autoinclusion.autoinclusion = false").select(['data.id']).where("day='2023-12-21'").distinct().toPandas()['id']))
len(s2 - s1)

745